In [21]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 1911, 1825, 1828], shape=(11314,))

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [30]:
idx_rand = np.random.randint(0, len(newsgroups_train.data), size=5)
# Recorremos el vector de valores aleatorios y medir la similitud 
for i in range(5):
    idx = idx_rand[i]
    print(f'\nDocumento original: {newsgroups_train.data[idx]}')
    print(f'Clase original: {newsgroups_train.target_names[y_train[idx]]}')
    
    # Medimos la similaridad coseno con todos los documentos de train
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    
    # Obtenemos los 5 documentos más similares
    mostsim = np.argsort(cossim)[::-1][1:6]
    print('\n############################################################################\n')
    # Mostramos los documentos más similares y sus clases
    for j in mostsim:
        #print(f'Similar: {newsgroups_train.data[j]}')
        print(f'Clase similar: {newsgroups_train.target_names[y_train[j]]}')


Documento original: I am in need of the Driver for the Bernoulli Cartridge on a DataFrame
XP60+B.  The hard disk on the system got fried and I haven't been able to
locate the original disks.  If anyone has it or know where I can get this
please let me know via e-mail.

Thanks in advance.

Wayne Lyle

-- 
Clase original: comp.sys.mac.hardware

############################################################################

Clase similar: comp.sys.mac.hardware
Clase similar: comp.sys.mac.hardware
Clase similar: rec.sport.hockey
Clase similar: comp.sys.ibm.pc.hardware
Clase similar: comp.sys.mac.hardware

Documento original: Some time back I asked for software recommendations to  allow me to
run X from my PC at home to my Sparc at the office.

Many thanks for all replies, the majority of people  recommended
PCXremote from NCD. I received it yesterday  and installed it on
my Sparc and PC with only one hitch. 
The UNIX install consists of copying 2 files into some local bin directory. 
On the

#### Comentario 

Se observa que en algunos casos a partir del 3ro o 4to documento con mayor similitud de coseno la clase es completamente distinta esto por que estamos tomando frecuencias relativas para el vectoriza

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [32]:
# Prueba con diferentes parámetros del vectorizador
vectorizers = [
    TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.8, min_df=5),
    CountVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.8, min_df=5)
]

models = [
    MultinomialNB(),
    ComplementNB()
]

best_score = 0
best_config = None

for vect in vectorizers:
    X_train = vect.fit_transform(newsgroups_train.data)
    X_test = vect.transform(newsgroups_test.data)
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred, average='macro')
        print(f"Vectorizer: {vect.__class__.__name__}, Model: {model.__class__.__name__}, F1 macro: {score:.4f}")
        if score > best_score:
            best_score = score
            best_config = (vect, model)

print("\nMejor configuración:")
print(f"Vectorizer: {best_config[0]}")
print(f"Modelo: {best_config[1]}")
print(f"Mejor F1 macro: {best_score:.4f}")

Vectorizer: TfidfVectorizer, Model: MultinomialNB, F1 macro: 0.6503
Vectorizer: TfidfVectorizer, Model: ComplementNB, F1 macro: 0.6915
Vectorizer: CountVectorizer, Model: MultinomialNB, F1 macro: 0.6167
Vectorizer: CountVectorizer, Model: ComplementNB, F1 macro: 0.6346

Mejor configuración:
Vectorizer: TfidfVectorizer(max_df=0.8, min_df=5, ngram_range=(1, 2), stop_words='english')
Modelo: ComplementNB()
Mejor F1 macro: 0.6915


**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [ ]:
# Transponemos la matriz documento-término (X_train)
TfidfVectorizer
X_terms = X_train.T  # Ahora cada fila es un término y cada columna un documento
print(tfidfvect.vocabulary_['computer'])
# Elegimos 5 palabras manualmente (debe existir en el vocabulario)
palabras = ['car', 'computer', 'god', 'space', 'windows']
indices = [tfidfvect.vocabulary_[p] for p in palabras]
print(indices)
for idx, palabra in zip(indices, palabras):
    # Calculamos la similaridad coseno de la palabra con todas las demás
    sim = cosine_similarity(X_terms[idx], X_terms)[0]
    # Obtenemos los 5 términos más similares (excluyendo la palabra misma)
    most_similar = np.argsort(sim)[::-1][1:6]
    print(f"\nPalabra: '{palabra}'")
    print("Palabras más similares:")
    for i in most_similar:
        print(f"  {idx2word[i]} (sim={sim[i]:.3f})")

28940
[25775, 28940, 43842, 84097, 96760]

Palabra: 'car'
Palabras más similares:
  caper (sim=0.430)
  8utj0ht654 (sim=0.382)
  85t21m (sim=0.355)
  biomedical (sim=0.348)
  cmhnet (sim=0.332)


IndexError: index (28940) out of range

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

